1. Example of calculating the similarity between words using pre-trained wiki word vectors

In [8]:
from gensim.models.keyedvectors import KeyedVectors
corpus=KeyedVectors.load_word2vec_format("wiki.en.vec")
corpus.most_similar("man")


C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[('woman', 0.6511167287826538),
 ('man—', 0.5554498434066772),
 ('man/one', 0.5516811609268188),
 ('stranger', 0.5505865216255188),
 ('boy', 0.5456836223602295),
 ('woman—a', 0.5385604500770569),
 ('spider', 0.5372462272644043),
 ('girl', 0.5341853499412537),
 ('man—a', 0.5321135520935059),
 ('man—and', 0.5311824083328247)]

In [68]:
type(corpus)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [24]:
corpus.similarity('vegetable','horse')

0.23583984923717163

In [29]:
corpus.similarity('donkey','horse')

0.5242442005632283

2. Classification of HScode.xlsx file accoring to the size of item categories 

Find a code in the bigger to smaller categories.

In [153]:
import pandas as pd
import numpy as np

hs=pd.read_excel('2016(hsk)_data_1.1.xlsx',sheet_name='HS코드집',converters={'HS코드':str,'코드명(품명)':str})
hs=hs.sort_index()
hs.columns=['HS코드','코드명(품명)']

print(hs.shape) 

hs[:3]

(17966, 2)


,HS코드,코드명(품명)
0,0101,"Live horses, asses, mules and hinnies."
1,01012,Horses :
2,010121,Pure-bred breeding animals


In [159]:
arr=np.array(hs)
hscode = arr[:,0] #HS코드
item = arr[:,1] #코드명(품명)

#print(hscode)

type(item[0])


<class 'str'>

In [160]:
def split_by_n(string,offset):
    return string[:offset]

#print(split_by_offset('0234921',4))

In [161]:
hold='0000'
itemlist=[]
for code in hscode:
    if(hold < split_by_n(code,4)):
        itemlist.append(item[list(hscode).index(code)])
        hold=split_by_n(code,4)

print(len(itemlist))

for i in range(0,20):
    print(itemlist[i])

#print(itemlist) #: 가장 큰 범주의 상품들이 출력된다. 
#ex) HS코드번호 0101, 0102, 0103 ...0204, 020500, 0206 ... 의 상품명

1224
Live horses, asses, mules and hinnies.
Live bovine animals.
Live swine.
Live sheep and goats.
Live poultry, that is to say, fowls of the species Gallus domesticus, ducks, geese, turkeys and guinea fowls.
Other live animals.
Meat of bovine animals, fresh or chilled.
Meat of bovine animals, frozen.
Meat of swine, fresh, chilled or frozen.
Meat of sheep or goats, fresh, chilled or frozen.
Meat of horses, asses, mules or hinnies, fresh, chilled or frozen.
Edible offal of bovine animals, swine, sheep, goats, horses, asses, mules or hinnies, fresh, chilled or frozen.
Meat and edible offal, of the poultry of heading 01.05, fresh, chilled or frozen.
Other meat and edible meat offal, fresh, chilled or frozen.
Pig fat, free of lean meat, and poultry fat, not rendered or otherwise extracted, fresh, chilled, frozen, salted, in brine, dried or smoked.
Meat and edible meat offal, salted, in brine, dried or smoked; edible flours and meals of meat or meat offal.
Live fish.
Fish, fresh or chilled,

 3. Example of calculating the similarity between phrases using nltk and tf-idf

In [96]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
#여러가지 이유로 변화된 단어의 접미사나 어미를 제거하여 같은 의미를 가지는 형태소의 실제 형태를 동일하게 만든다.

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


print (cosine_sim('a little bird', 'a little bird'))
print (cosine_sim('a little bird', 'a little bird chirps'))
print (cosine_sim('a little bird', 'a big dog barks'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
0.9999999999999998
0.7092972666062738
0.0


In [124]:
import operator

d={}

for item in itemlist:
    d[cosine_sim('MIXED SEASONINGS POWDER', item)]=item    
        
sorted_d = sorted(d.items(), key=operator.itemgetter(0),reverse=True)        

In [125]:
from pprint import pprint

In [144]:
print(hscode[list(item).index(sorted_d[0][1])]) #가장 유사한 상품의 코드번호

#print(sorted_d[0][1]) #가장 유사한 상품명

pprint(sorted_d) #list of dictionary, 유사도 내림차순

2103
[(0.2716512395696402,
  'Sauces and preparations therefor; mixed condiments and mixed seasoning; '
  'mustard flour and meal and prepared mustard.'),
 (0.2605556710562624, 'Propellent powders.'),
 (0.24129913647238907,
  'Mixed alkylbenzenes and mixed alkylnaphthalenes, other than those of '
  'heading 2707 or 2902.'),
 (0.20199309249791833, 'Aluminium powders and flakes.'),
 (0.17077611319011649, 'Zinc dust, powders and flakes.'),
 (0.13328417694886524,
  'Animal or vegetable fertilisers, whether or not mixed together or '
  'chemically treated; fertilisers produced by the mixing or chemical '
  'treatment of animal or vegetable products.'),
 (0.1322483762954801,
  'Machinery for sorting, screening, separating, washing, crushing, grinding, '
  'mixing or kneading earth, stone, ores or other mineral substances, in solid '
  '(including powder or paste) form; machinery for agglomerating, shaping or '
  'moulding solid mineral fuels, ceramic paste, unhardened cements, plastering '
 